In [173]:
import pandas as pd
from pandas import option_context
import numpy as np

import re
import string

### This is the main notebook for cleaning the app dataframe

In [174]:
df = pd.read_csv('04-data/scraped_app_data.csv')
df.info()

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46031 entries, 0 to 46030
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     46031 non-null  object 
 1   description               46025 non-null  object 
 2   descriptionHTML           46025 non-null  object 
 3   summary                   46009 non-null  object 
 4   summaryHTML               46009 non-null  object 
 5   installs                  46021 non-null  object 
 6   minInstalls               46021 non-null  float64
 7   score                     45932 non-null  float64
 8   ratings                   45877 non-null  float64
 9   reviews                   45932 non-null  float64
 10  histogram                 46021 non-null  object 
 11  price                     46021 non-null  float64
 12  free                      46021 non-null  object 
 13  currency                  46021 non-null  object 
 14  sale  

In [175]:
df = df.drop(columns = ['descriptionHTML',
                       'summaryHTML',
                       'recentChangesHTML',
                       'developerEmail',
                       'developerWebsite',
                       'developerAddress',
                       'saleTime',
                       'originalPrice',
                       'saleText'])

In [176]:
df['year'] = pd.DatetimeIndex(df['released']).year

In [177]:
df.sample(2)

,title,description,summary,installs,minInstalls,score,ratings,reviews,histogram,price,...,containsAds,released,updated,version,recentChanges,comments,editorsChoice,appId,url,year
601,ASSIST CARD,ASSIST CARD is the largest travel assistance o...,ASSIST CARD is the largest travel assistance o...,"500,000+",500000.0,2.916031,1304.0,722.0,"[468, 139, 139, 149, 409]",0.0,...,False,"Aug 19, 2014",1.613750e+09,4.2.71,We continue to improve the quality of our serv...,"['Can\'t get in touch with what they call an ""...",False,com.assistcard.assistcard,https://play.google.com/store/apps/details?id=...,2014.0
44512,US Police Robot Transform Helicopter Wars,US Police Robot Transform Helicopter Wars,US Police Robot Transform Helicopter Wars,"1,000,000+",1000000.0,4.127778,1788.0,555.0,"[268, 99, 39, 109, 1273]",0.0,...,True,"Mar 7, 2018",1.596031e+09,Varies with device,Minor bugs fixed,['This is amazing game I like it very much I l...,False,com.trnz.us.police.robot.helicopter.war,https://play.google.com/store/apps/details?id=...,2018.0


In [178]:
df['description'] = df['description'].astype(str)

In [179]:
def clean_text(text):
    '''Make text lowercase, remove punctuation, remove emojis, etc'''
    text = text.lower()
    text = re.sub(r"<[^>]*>", "", text)
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub("[^a-zA-Z0-9 -]","",text)
    
    return text

df['description_clean'] = df.description.map(clean_text)

In [180]:
# Remove rows that do not have a value for ratings / score
df = df[df['ratings'].notna()]

In [181]:
# Drop duplicates
df = df.drop_duplicates(subset=['appId', 'title'], keep='last')

In [182]:
# Clean star historgram and separate into new columns
df['hist_clean'] = df['histogram'].apply(lambda x: x[1:-1].split(','))
df['hist_clean'] = df['hist_clean'].apply(lambda y: [int(x) for x in y])

In [183]:
df[['star_1','star_2','star_3','star_4','star_5']] = pd.DataFrame(df.hist_clean.tolist(), index= df.index)

In [184]:
# Remove non-english apps
df = df[df['appId'] != 'com.syc.librototal.El_Libro_Total']

In [185]:
# Change data type to bool for analysis 
df['editorsChoice'] = df['editorsChoice'].astype(str)
df['editorsChoice'] = np.where(df['editorsChoice'] == 'True',1,0)

df['free'] = df['free'].astype(str)
df['free'] = np.where(df['free'] == 'True',1,0)

df['containsAds'] = df['containsAds'].astype(str)
df['containsAds'] = np.where(df['containsAds'] == 'True',1,0)

In [187]:
df

,title,description,summary,installs,minInstalls,score,ratings,reviews,histogram,price,...,appId,url,year,description_clean,hist_clean,star_1,star_2,star_3,star_4,star_5
0,World War 2: Offline Strategy,Command allies in 25 epic World War 2 locatio...,Historic events of World War 2,"1,000,000+",1000000.0,4.147644,19080.0,5581.0,"[1908, 759, 1288, 3776, 11349]",0.0,...,com.skizze.wwii,https://play.google.com/store/apps/details?id=...,2018.0,command allies in 25 epic world war 2 locatio...,"[1908, 759, 1288, 3776, 11349]",1908,759,1288,3776,11349
1,AndroXLS editor for XLS sheets,AndroXLS is an android app to edit XLS spreads...,Editor for xls spreadsheets that supports Libr...,"5,000,000+",5000000.0,2.973147,9215.0,3770.0,"[3762, 554, 702, 564, 3633]",0.0,...,com.entertain.androxls,https://play.google.com/store/apps/details?id=...,2016.0,androxls is an android app to edit xls spreads...,"[3762, 554, 702, 564, 3633]",3762,554,702,564,3633
2,SoundSeeder -Play music simultaneously and in ...,SoundSeeder syncs up your music playback on mu...,"Synced playback of MP3, Internet Radio and Goo...","1,000,000+",1000000.0,3.747968,7347.0,2490.0,"[1523, 328, 497, 1125, 3874]",0.0,...,com.kattwinkel.android.soundseeder.player,https://play.google.com/store/apps/details?id=...,2013.0,soundseeder syncs up your music playback on mu...,"[1523, 328, 497, 1125, 3874]",1523,328,497,1125,3874
3,LibreOffice & OpenOffice document reader | ODF,<h2><b>View and modify documents created using...,Document reader & file editor for Libreoffice ...,"5,000,000+",5000000.0,3.882280,32406.0,8381.0,"[5253, 1368, 2626, 5852, 17307]",0.0,...,at.tomtasche.reader,https://play.google.com/store/apps/details?id=...,2010.0,view and modify documents created using libreo...,"[5253, 1368, 2626, 5852, 17307]",5253,1368,2626,5852,17307
4,"All PDF - PDF Reader, PDF Viewer & PDF Converter","PDF Reader for android, PDFs converter free is...","PDF Reader, PDF Reader 2021 Converter. Reduce ...","1,000,000+",1000000.0,4.376968,10073.0,4418.0,"[684, 257, 714, 1338, 7080]",0.0,...,tz.co.wadau.allpdf,https://play.google.com/store/apps/details?id=...,2018.0,pdf reader for android pdfs converter free is ...,"[684, 257, 714, 1338, 7080]",684,257,714,1338,7080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46026,Bikini Bottom Maps for Minecraft,Play the best maps of Bikini Bottom with this ...,Play the best maps of Bikini Bottom for Minecr...,"1,000,000+",1000000.0,3.960000,294.0,135.0,"[59, 8, 11, 20, 196]",0.0,...,com.wolfgames.bikinibottom,https://play.google.com/store/apps/details?id=...,2020.0,play the best maps of bikini bottom with this ...,"[59, 8, 11, 20, 196]",59,8,11,20,196
46027,PokeCraft : Monsters Mod,The <b>PokeCraft Monsters Mod</b> 🔥🔥🔥 for Mine...,Play to PokeCraft : Monsters Mod,"500,000+",500000.0,3.913295,3454.0,1820.0,"[829, 29, 89, 169, 2338]",0.0,...,li.pokemonsters.mcpe,https://play.google.com/store/apps/details?id=...,2020.0,the pokecraft monsters mod for minecraft is a...,"[829, 29, 89, 169, 2338]",829,29,89,169,2338
46028,TongitsXtreme,Enjoy this exciting Filipino card game called ...,Enjoy the exciting Filipino card game called T...,"5,000,000+",5000000.0,4.348361,14605.0,3832.0,"[1267, 319, 1037, 1416, 10566]",0.0,...,com.bandoy.tongitsXtreme,https://play.google.com/store/apps/details?id=...,2016.0,enjoy this exciting filipino card game called ...,"[1267, 319, 1037, 1416, 10566]",1267,319,1037,1416,10566
46029,Play Tube,---------- Play Tube Music ---------\r\n\r\nTH...,Watch many videos easily,"1,000,000+",1000000.0,4.024169,3304.0,1001.0,"[579, 89, 229, 179, 2228]",0.0,...,com.funnyapplab.videoplayer,https://play.google.com/store/apps/details?id=...,2018.0,---------- play tube music ---------this app d...,"[579, 89, 229, 179, 2228]",579,89,229,179,2228


In [186]:
df.groupby('editorsChoice').count()

,title,description,summary,installs,minInstalls,score,ratings,reviews,histogram,price,...,appId,url,year,description_clean,hist_clean,star_1,star_2,star_3,star_4,star_5
editorsChoice,,,,,,,,,,,,,,,,,,,,,
0,45565,45565,45549,45565,45565,45565,45565,45565,45565,45565,...,45562,45562,45315,45565,45565,45565,45565,45565,45565,45565
1,282,282,282,282,282,282,282,282,282,282,...,282,282,280,282,282,282,282,282,282,282


In [198]:
# Reassign a few apps
df.loc[(df['contentRating'] == 'Adults only 18+') |
      (df['contentRating'] == 'Unrated'),
      ['contentRating']] = 'Mature 17+'

In [199]:
df.groupby('contentRating').count()

,title,description,summary,installs,minInstalls,score,ratings,reviews,histogram,price,...,appId,url,year,description_clean,hist_clean,star_1,star_2,star_3,star_4,star_5
contentRating,,,,,,,,,,,,,,,,,,,,,
Everyone,35318,35318,35303,35318,35318,35318,35318,35318,35318,35318,...,35316,35316,35097,35318,35318,35318,35318,35318,35318,35318
Everyone 10+,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,...,2021,2021,2010,2021,2021,2021,2021,2021,2021,2021
Mature 17+,1737,1737,1736,1737,1737,1737,1737,1737,1737,1737,...,1737,1737,1732,1737,1737,1737,1737,1737,1737,1737
Teen,6771,6771,6771,6771,6771,6771,6771,6771,6771,6771,...,6770,6770,6756,6771,6771,6771,6771,6771,6771,6771


In [200]:
df.to_csv('04-data/preprocessed_app_data.csv',index=False)